# Hoja de trabajo 7
Jose Daniel Gomez Cabrera 21429

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import confusion_matrix
import time

# Carga de datos

In [2]:
train = pd.read_csv("./train.csv")


In [ ]:
# Manejo de valores faltantes
train.fillna(0, inplace=True)

# Creación de variable categórica
train['tipoDeCasa'] = pd.cut(train['SalePrice'], bins=[0, 145000, 205000, 410000], labels=[1, 2, 3])

# Conversión de variables categóricas
train = pd.get_dummies(train, columns=train.select_dtypes(include='object').columns)

In [ ]:
# División de datos en conjunto de entrenamiento y prueba
train, test = train_test_split(train, test_size=0.3, random_state=123)

# Selección de características
frstselect = train.iloc[:, [1, 2, 3, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66]]
scndselect = train.iloc[:, [1, 3, 4, 17, 18, 19, 20, 26, 34, 36, 37, 38, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 55, 57, 60, 62, 63, 67, 68, 69, 70, 71, 72, 76, 77, 78, 82]]

In [ ]:
# Modelos de SVM
models = [
    svm.SVC(kernel='linear', C=2**5),
    svm.SVC(kernel='linear', C=0.5),
    svm.SVC(kernel='linear', C=2**-5),
    svm.SVC(kernel='rbf', gamma=0.005),
    svm.SVC(kernel='rbf', gamma=0.05),
    svm.SVC(kernel='rbf', gamma=2**-5),
    svm.SVC(kernel='poly', gamma=1, coef0=1, degree=8),
    svm.SVC(kernel='poly', gamma=5, coef0=1),
    svm.SVC(kernel='poly', gamma=2**-5, coef0=1)
]

In [ ]:
    # Entrenamiento de modelos
for model in models:
    model.fit(scndselect.drop(columns=['tipoDeCasa']), scndselect['tipoDeCasa'])

In [ ]:
# Predicción y evaluación de modelos
predictions = {}
for model in models:
    start_time = time.process_time()
    predictions[model] = model.predict(test.iloc[:, 1:])
    elapsed_time = time.process_time() - start_time
    print(f"Prediction time for {model.kernel} kernel: {elapsed_time:.4f} seconds")
    cm = confusion_matrix(test['tipoDeCasa'], predictions[model])
    print(cm)